In [12]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random
import shutil

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
import time


device = torch.device('cuda')

In [6]:
# Input your own directory

file_dir = 'D:/thon/DL/zindi'
img_dir = file_dir + '/Images'

In [9]:
train_df = pd.read_csv(file_dir + "/Train.csv")
test_df = pd.read_csv(file_dir + '/Test.csv')
train_labels = train_df["Label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [13]:
### You don't need to do it twice

# Generate new 'train', 'test' image folders in your file_directory

os.makedirs(img_dir + '/train')
os.makedirs(img_dir + '/test')

# Set train and test directories

train_dir = img_dir + '/train'
test_dir = img_dir + '/test'

### To move files according to train.csv and test.csv's 'Image_id' column

train_file_name = train_df["Image_id"]
train_file_label = train_df["Label"]
test_file_name = test_df["Image_id"]

# Move train images from 'Images' folder to 'train' folder

for i in train_file_name:
    file_source = img_dir + f'/{i}'
    file_destination = train_dir
    shutil.move(file_source, file_destination)


# Move test images from 'Images' folder to 'test' folder

for i in test_file_name:
    file_source = img_dir + f'/{i}'
    file_destination = test_dir
    shutil.move(file_source, file_destination)

In [14]:
train_jpg = sorted(glob(train_dir +'/*.jpg'))
test_jpg = sorted(glob(test_dir + '/*.jpg'))

In [16]:
img_size = 224 # input size for efficientnet_b0 : 224 * 224

def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (img_size, img_size))
    return img

In [17]:
train_imgs = [img_load(m) for m in tqdm(train_jpg)]
test_imgs = [img_load(n) for n in tqdm(test_jpg)]

100%|██████████| 1080/1080 [00:15<00:00, 70.10it/s]


In [18]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()
        img = transforms.ToTensor()(img)
        if self.mode=='test':
            pass
        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [19]:
batch_size = 32
epochs = 25

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [10]:
def score_function(real, pred):
    score = roc_auc_score(real, pred, average="macro")
    return score

model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 



best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_auc = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    AUC : {train_auc:.5f}')

epoch : 1/25    time : 19s/464s
TRAIN    loss : 0.46129    f1 : 0.92170
epoch : 2/25    time : 14s/316s
TRAIN    loss : 0.04454    f1 : 0.98888
epoch : 3/25    time : 14s/304s
TRAIN    loss : 0.02314    f1 : 0.99197
epoch : 4/25    time : 14s/290s
TRAIN    loss : 0.05213    f1 : 0.98456
epoch : 5/25    time : 14s/278s
TRAIN    loss : 0.02688    f1 : 0.99321
epoch : 6/25    time : 14s/263s
TRAIN    loss : 0.00886    f1 : 0.99753
epoch : 7/25    time : 14s/251s
TRAIN    loss : 0.00214    f1 : 0.99938
epoch : 8/25    time : 14s/238s
TRAIN    loss : 0.00371    f1 : 0.99815
epoch : 9/25    time : 14s/225s
TRAIN    loss : 0.01079    f1 : 0.99629
epoch : 10/25    time : 14s/210s
TRAIN    loss : 0.02092    f1 : 0.99382
epoch : 11/25    time : 14s/196s
TRAIN    loss : 0.00277    f1 : 0.99938
epoch : 12/25    time : 14s/183s
TRAIN    loss : 0.03383    f1 : 0.99074
epoch : 13/25    time : 14s/170s
TRAIN    loss : 0.02401    f1 : 0.99321
epoch : 14/25    time : 14s/155s
TRAIN    loss : 0.00435    

In [11]:
model.eval()
f_pred = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [12]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [13]:
submission = pd.read_csv("./zindi/SampleSubmission.csv")
submission["Label"] = f_result
submission

,Image_id,Label
0,id_00exusbkgzw1b.jpg,0
1,id_03dqinf6w0znv.jpg,0
2,id_046yl0cxn3ybz.jpg,1
3,id_04athdtx2abyg.jpg,0
4,id_062aauf9e9jk0.jpg,0
...,...,...
1075,id_zv5fvjnakvf1r.jpg,1
1076,id_zvpikh1z30arn.jpg,0
1077,id_zypilwkudljyz.jpg,0
1078,id_zz9lwehh5sxdp.jpg,1


In [14]:
submission.to_csv('Submission_v2.csv', index = False)